# transfor_sk_keypts
This script is for the tansformation of deterministic method.
For alignment, this scipt will generate the transformed keypoints.csv in addition to skeletons.

## Part 1
Transform skeletons by translation, rotate, scaling, and half-resize.

Change root

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
def resize_half_image(
    img,
    keypoints=None,
    scale_factor=1.5,
    resize_upper=True,
    horizontal=False,
):
    """
    對圖片的一半進行比例改變，並同步更新 keypoints 座標。

    模式：
        horizontal = False（預設）：做「上下」變形
            - resize_upper = True  → 改上半部分（只影響 y）
            - resize_upper = False → 改下半部分（只影響 y）
        horizontal = True：做「左右」變形
            - resize_upper = True  → 改左半部分（只影響 x）
            - resize_upper = False → 改右半部分（只影響 x）

    參數:
        img: 輸入圖片 (BGR格式)
        keypoints: (可選) numpy array，shape = (N, 2)，每列為 [x, y]
        scale_factor: 縮放比例 (例如 1.5 表示拉伸1.5倍，0.7 表示壓縮到0.7倍)
        resize_upper: 見上方說明
        horizontal: False=上下半，True=左右半

    返回:
        若 keypoints 為 None: result_img
        否則: (result_img, transformed_kps)
    """
    height, width = img.shape[:2]

    transformed_kps = None
    kps = None
    if keypoints is not None:
        kps = keypoints.astype(np.float32).copy()
        x = kps[:, 0]
        y = kps[:, 1]

    # ========================
    # 垂直方向（上下半）變形
    # ========================
    if not horizontal:
        mid_point = height // 2

        if resize_upper:
            # 改變上半部分
            upper_half = img[0:mid_point, :]
            lower_half = img[mid_point:, :]

            # 縮放上半部分（改高度）
            new_upper_height = int(upper_half.shape[0] * scale_factor)
            upper_half_resized = cv2.resize(
                upper_half,
                (width, new_upper_height),
                interpolation=cv2.INTER_LINEAR
            )

            # 組合
            total_height = new_upper_height + lower_half.shape[0]
            result_img = np.zeros((total_height, width, 3), dtype=np.uint8)
            result_img[0:new_upper_height, :] = upper_half_resized
            result_img[new_upper_height:, :] = lower_half

            if kps is not None:
                # 上半部: y < mid_point
                mask_upper = y < mid_point
                # 下半部: y >= mid_point
                mask_lower = ~mask_upper

                # 上半部 y' = y * scale_factor
                y[mask_upper] = y[mask_upper] * scale_factor
                # 下半部 y' = new_upper_height + (y - mid_point)
                y[mask_lower] = new_upper_height + (y[mask_lower] - mid_point)

                transformed_kps = np.stack([x, y], axis=1)

        else:
            # 改變下半部分
            upper_half = img[0:mid_point, :]
            lower_half = img[mid_point:, :]

            # 縮放下半部分（改高度）
            new_lower_height = int(lower_half.shape[0] * scale_factor)
            lower_half_resized = cv2.resize(
                lower_half,
                (width, new_lower_height),
                interpolation=cv2.INTER_LINEAR
            )

            # 組合
            total_height = upper_half.shape[0] + new_lower_height
            result_img = np.zeros((total_height, width, 3), dtype=np.uint8)
            result_img[0:upper_half.shape[0], :] = upper_half
            result_img[upper_half.shape[0]:, :] = lower_half_resized

            if kps is not None:
                # 上半部: y < mid_point (不變)
                mask_upper = y < mid_point
                # 下半部: y >= mid_point (縮放)
                mask_lower = ~mask_upper

                # 上半部 y' = y
                # 下半部 y' = mid_point + (y - mid_point) * scale_factor
                y[mask_lower] = mid_point + (y[mask_lower] - mid_point) * scale_factor

                transformed_kps = np.stack([x, y], axis=1)

    # ========================
    # 水平方向（左右半）變形
    # ========================
    else:
        mid_point = width // 2

        if resize_upper:
            # 改變左半部分
            left_half = img[:, 0:mid_point]
            right_half = img[:, mid_point:]

            # 縮放左半部分（改寬度）
            new_left_width = int(left_half.shape[1] * scale_factor)
            left_half_resized = cv2.resize(
                left_half,
                (new_left_width, height),
                interpolation=cv2.INTER_LINEAR
            )

            # 組合
            total_width = new_left_width + right_half.shape[1]
            result_img = np.zeros((height, total_width, 3), dtype=np.uint8)
            result_img[:, 0:new_left_width] = left_half_resized
            result_img[:, new_left_width:] = right_half

            if kps is not None:
                # 左半部: x < mid_point
                mask_left = x < mid_point
                # 右半部: x >= mid_point
                mask_right = ~mask_left

                # 左半部 x' = x * scale_factor
                x[mask_left] = x[mask_left] * scale_factor
                # 右半部 x' = new_left_width + (x - mid_point)
                x[mask_right] = new_left_width + (x[mask_right] - mid_point)

                transformed_kps = np.stack([x, y], axis=1)

        else:
            # 改變右半部分
            left_half = img[:, 0:mid_point]
            right_half = img[:, mid_point:]

            # 縮放右半部分（改寬度）
            new_right_width = int(right_half.shape[1] * scale_factor)
            right_half_resized = cv2.resize(
                right_half,
                (new_right_width, height),
                interpolation=cv2.INTER_LINEAR
            )

            # 組合
            total_width = left_half.shape[1] + new_right_width
            result_img = np.zeros((height, total_width, 3), dtype=np.uint8)
            result_img[:, 0:left_half.shape[1]] = left_half
            result_img[:, left_half.shape[1]:] = right_half_resized

            if kps is not None:
                # 左半部: x < mid_point (不變)
                mask_left = x < mid_point
                # 右半部: x >= mid_point (縮放)
                mask_right = ~mask_left

                # 左半部 x' = x
                # 右半部 x' = mid_point + (x - mid_point) * scale_factor
                x[mask_right] = mid_point + (x[mask_right] - mid_point) * scale_factor

                transformed_kps = np.stack([x, y], axis=1)

    if keypoints is None:
        return result_img
    else:
        return result_img, transformed_kps


In [ ]:
import random
import cv2
import numpy as np

def scale_image(
    img,
    keypoints=None,             # 新增：keypoints, shape = (N, 2)
    scale_percent=80,         # None 表示「沒特別指定」
    mode=None,                  # mode = 0 or 1，用來決定隨機區間
    border_percent_when_upscale=10,
    pad_value=0,
):
    """
    縮放圖片並加上 padding，並同步更新 keypoints 座標。

    - 有給 scale_percent (>0)：直接用這個數字
    - 沒給 scale_percent：
        * mode = 0 → 在 [80, 99] 之間隨機
        * mode = 1 → 在 [101, 120] 之間隨機
        * 其他 → 不縮放 (=100)
    - scale_percent < 100：縮小後貼到「原圖大小」的畫布中(四周是 padding)
      => keypoints 先乘縮放比例，再加置中偏移
    - scale_percent > 100：放大後貼到更大的畫布中(四周是 padding)
      => keypoints 先乘縮放比例，再加邊框偏移

    回傳：
        若 keypoints 為 None: canvas
        否則: (canvas, transformed_kps)
    """
    h, w = img.shape[:2]

    # ---------- 決定 scale_percent ----------
    if scale_percent is None or scale_percent <= 0:
        if mode == 0:
            scale_percent = random.randint(80, 99)
        elif mode == 1:
            scale_percent = random.randint(101, 120)
        else:
            # 沒給 mode 的 fallback：不縮放
            scale_percent = 100

    # 縮放比例
    s = float(scale_percent) / 100.0

    # 1. 計算縮放後大小
    new_w = int(w * s)
    new_h = int(h * s)

    # 避免 new_w 或 new_h 變成 0
    new_w = max(new_w, 1)
    new_h = max(new_h, 1)

    # 2. 先做縮放
    resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

    transformed_kps = None
    if keypoints is not None:
        kps = keypoints.astype(np.float32).copy()
        x = kps[:, 0]
        y = kps[:, 1]

        # 先做相同的縮放（以左上角為原點）
        x_scaled = x * s
        y_scaled = y * s

    # ---------- 情況一：縮小或等大（≤100） ----------
    if scale_percent <= 100:
        # 建立跟原圖同大小的畫布
        if img.ndim == 2:  # 灰階
            canvas = np.full((h, w), pad_value, dtype=img.dtype)
        else:              # 彩色
            canvas = np.full((h, w, img.shape[2]), pad_value, dtype=img.dtype)

        # 置中貼上縮小後圖片
        x_offset = (w - new_w) // 2
        y_offset = (h - new_h) // 2
        canvas[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = resized

        if keypoints is not None:
            # keypoints 也要加上同樣的置中偏移
            x_new = x_scaled + x_offset
            y_new = y_scaled + y_offset
            transformed_kps = np.stack([x_new, y_new], axis=1)

    # ---------- 情況二：放大 (>100) ----------
    else:
        # 放大時再加一圈 padding，避免放大後畫面剛好貼邊
        border_x = int(new_w * border_percent_when_upscale / 100)
        border_y = int(new_h * border_percent_when_upscale / 100)

        canvas_w = new_w + 2 * border_x
        canvas_h = new_h + 2 * border_y

        if img.ndim == 2:
            canvas = np.full((canvas_h, canvas_w), pad_value, dtype=img.dtype)
        else:
            canvas = np.full((canvas_h, canvas_w, img.shape[2]), pad_value, dtype=img.dtype)

        # 讓放大的圖置中（其實是加邊框）
        x_offset = border_x
        y_offset = border_y
        canvas[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = resized

        if keypoints is not None:
            # keypoints 加邊框偏移
            x_new = x_scaled + x_offset
            y_new = y_scaled + y_offset
            transformed_kps = np.stack([x_new, y_new], axis=1)

    if keypoints is None:
        return canvas
    else:
        return canvas, transformed_kps


In [ ]:
import os
import csv
from tqdm import tqdm
import cv2
import numpy as np


def batch_process_images(
    input_dir,
    output_base_dir,
    transformations,
    keypoint_csv_path=None,
    keypoint_output_dir=None,
):
    """
    批次處理圖片 (+ 選擇性處理 keypoints)

    參數:
        input_dir: 輸入圖片目錄（骨架 PNG）
        output_base_dir: 輸出圖片的基礎目錄
        transformations: 變形操作字典
        keypoint_csv_path: (可選) 原始 keypoints CSV 路徑
            - CSV 中的 filename 是「原始彩圖檔名（含副檔名）」。
        keypoint_output_dir: (可選) 變形後 CSV 的輸出資料夾
    """

    # 確保輸入目錄存在
    if not os.path.exists(input_dir):
        print(f"錯誤：輸入目錄不存在: {input_dir}")
        return

    # -------------------------
    # 1) 準備圖片輸出目錄
    # -------------------------
    os.makedirs(output_base_dir, exist_ok=True)

    dataset_dirs = {}
    for dataset_name in transformations.keys():
        dataset_path = os.path.join(output_base_dir, dataset_name)
        os.makedirs(dataset_path, exist_ok=True)
        dataset_dirs[dataset_name] = dataset_path
        print(f"創建資料集目錄: {dataset_path}")

    # -------------------------
    # 2) 讀取 keypoints CSV (如果有給)
    # -------------------------
    keypoints_by_stem = {}  # key = 不含附檔名的檔名（stem）
    kp_header = None
    kp_pairs = []  # [(x_col, y_col), ...]

    if keypoint_csv_path is not None and os.path.exists(keypoint_csv_path):
        with open(keypoint_csv_path, "r", newline="") as f:
            reader = csv.DictReader(f)
            kp_header = reader.fieldnames

            if kp_header is None or "filename" not in kp_header:
                raise ValueError("CSV 檔必須包含 'filename' 欄位")

            # 根據欄位順序自動推 keypoint x/y 成對
            cols = kp_header[1:]  # 除掉 filename
            if len(cols) % 2 != 0:
                raise ValueError("除了 filename 之外，CSV 欄位數應該是偶數 (x,y 成對)")

            for i in range(0, len(cols), 2):
                x_col = cols[i]
                y_col = cols[i + 1]
                kp_pairs.append((x_col, y_col))

            for row in reader:
                fname_with_ext = row["filename"]           # e.g. "000123.jpg" 或 "000123.png"
                stem = os.path.splitext(fname_with_ext)[0] # "000123"

                if stem in keypoints_by_stem:
                    # 如果真的同一個 stem 出現兩個不一樣的檔名，可以在這裡印警告
                    # print(f"警告：CSV 中有重複的 stem: {stem}")
                    pass

                keypoints_by_stem[stem] = row

        print(f"已讀取 keypoints CSV，共 {len(keypoints_by_stem)} 筆 (以 stem 作為 key)")

    # -------------------------
    # 3) 準備各 transform 對應的輸出 CSV writer
    # -------------------------
    kp_writers = {}
    kp_files = {}

    if kp_header is not None:
        if keypoint_output_dir is None:
            keypoint_output_dir = output_base_dir
        os.makedirs(keypoint_output_dir, exist_ok=True)

        for dataset_name in transformations.keys():
            csv_path = os.path.join(keypoint_output_dir, f"{dataset_name}.csv")
            f = open(csv_path, "w", newline="")
            writer = csv.DictWriter(f, fieldnames=kp_header)
            writer.writeheader()
            kp_files[dataset_name] = f
            kp_writers[dataset_name] = writer
            print(f"為 {dataset_name} 建立 keypoints CSV: {csv_path}")

    # -------------------------
    # 4) 找出所有圖片檔（骨架 PNG 等）
    # -------------------------
    image_extensions = [".png", ".jpg", ".jpeg", ".bmp", ".tiff", ".tif"]
    image_files = [
        file for file in os.listdir(input_dir)
        if any(file.lower().endswith(ext) for ext in image_extensions)
    ]

    print(f"\n找到 {len(image_files)} 張圖片")
    print(f"將生成 {len(image_files) * len(transformations)} 張變形圖片\n")

    # 統計資訊
    stats = {name: {"success": 0, "failed": 0} for name in transformations.keys()}

    # -------------------------
    # 5) 處理每張圖片
    # -------------------------
    try:
        for img_file in tqdm(image_files, desc="處理圖片"):
            img_path = os.path.join(input_dir, img_file)

            # 讀取圖片
            img = cv2.imread(img_path)
            if img is None:
                print(f"警告：無法讀取圖片 {img_file}")
                continue

            # 檔名拆分
            file_name_without_ext, file_ext = os.path.splitext(img_file)  # 骨架：e.g. "000123", ".png"

            # 準備對應的 keypoints（如果有）
            row = None
            keypoints_np = None

            if keypoints_by_stem:
                # 用「stem」來對應 CSV（原圖 .jpg/.png 都可以，但 stem 一樣）
                row = keypoints_by_stem.get(file_name_without_ext)
                if row is None:
                    print(f"警告：找不到 stem {file_name_without_ext} 對應的 keypoints，將只處理圖片")
                else:
                    kp_list = []
                    for x_col, y_col in kp_pairs:
                        x_val = float(row[x_col])
                        y_val = float(row[y_col])
                        kp_list.append([x_val, y_val])
                    keypoints_np = np.array(kp_list, dtype=np.float32)

            # 對每個 transformation 做處理
            for dataset_name, transform_info in transformations.items():
                try:
                    transform_func = transform_info["function"]
                    transform_params = transform_info.get("params", {})

                    # --- 呼叫變換函式 ---
                    if keypoints_np is not None:
                        out = transform_func(
                            img,
                            keypoints=keypoints_np,
                            **transform_params,
                        )
                        if isinstance(out, tuple) and len(out) == 2:
                            transformed_img, transformed_kps = out
                        else:
                            transformed_img = out
                            transformed_kps = None
                    else:
                        transformed_img = transform_func(img, **transform_params)
                        transformed_kps = None

                    # --- 存圖片 ---
                    output_filename = f"{file_name_without_ext}{file_ext}"
                    output_path = os.path.join(dataset_dirs[dataset_name], output_filename)
                    cv2.imwrite(output_path, transformed_img)

                    # --- 存 keypoints 到對應 CSV ---
                    if transformed_kps is not None and dataset_name in kp_writers and row is not None:
                        new_row = dict(row)  # 複製原本那一行

                        # 這裡維持原來 CSV 裡的 filename（原始彩圖檔名），
                        # 如果你想改成骨架檔名，也可以改：
                        # new_row["filename"] = output_filename

                        for idx, (x_col, y_col) in enumerate(kp_pairs):
                            x_new, y_new = transformed_kps[idx]
                            new_row[x_col] = float(x_new)
                            new_row[y_col] = float(y_new)

                        kp_writers[dataset_name].writerow(new_row)

                    stats[dataset_name]["success"] += 1

                except Exception as e:
                    print(f"錯誤：處理 {img_file} 的 {dataset_name} 變形時發生錯誤: {str(e)}")
                    stats[dataset_name]["failed"] += 1

    finally:
        # 關閉所有 CSV 檔案
        for f in kp_files.values():
            f.close()

    return stats


In [ ]:
# 定義變形操作設定
transformations = {
    'original_70': {
        'function': scale_image,
        'params': { 'scale_percent': 70 }
    },
    # 上半部分拉長
    'resize_upper_1.5': {
        'function': resize_half_image,
        'params': {'scale_factor': 1.5, 'resize_upper': True, 'horizontal': False}
    },
    # 上半部分壓縮
    'resize_upper_0.7': {
        'function': resize_half_image,
        'params': {'scale_factor': 0.7, 'resize_upper': True, 'horizontal': False}
    },
    # 下半部分拉長
    'resize_lower_1.3': {
        'function': resize_half_image,
        'params': {'scale_factor': 1.3, 'resize_upper': False, 'horizontal': False}
    },
    # 下半部分壓縮
    'resize_lower_0.5': {
        'function': resize_half_image,
        'params': {'scale_factor': 0.5, 'resize_upper': False, 'horizontal': False}
    },
    # 右半部分拉寬
    'resize_left_1.5': {
        'function': resize_half_image,
        'params': {'scale_factor': 1.5, 'resize_upper': True, 'horizontal': True}
    },
    # 右半部分縮窄
    'resize_left_0.7': {
        'function': resize_half_image,
        'params': {'scale_factor': 0.7, 'resize_upper': True, 'horizontal': True}
    },
    # 左半部分拉寬
    'resize_right_1.3': {
        'function': resize_half_image,
        'params': {'scale_factor': 1.3, 'resize_upper': False, 'horizontal': True}
    },
    # 左半部分縮窄
    'resize_right_0.5': {
        'function': resize_half_image,
        'params': {'scale_factor': 0.5, 'resize_upper': False, 'horizontal': True}
    },
}

# ==== 路徑設定（請改成你實際的路徑）====
root = ""
# 骨架圖片所在資料夾
input_dir = root+"/sk/original"

# 變形後圖片的輸出根目錄（裡面會自動建立各個 transform 子資料夾）
output_dir = root+"/sk"

# 對應原始骨架圖片的 keypoints CSV
keypoint_csv_path = root+"/sk/original.csv"

# 變形後 keypoints CSV 的輸出資料夾
keypoint_output_dir = root+"/sk"

# 執行批次處理 (如果 input_dir 存在)
if os.path.exists(input_dir):
    print(f"開始處理目錄: {input_dir}")
    stats = batch_process_images(
        input_dir=input_dir,
        output_base_dir=output_dir,
        transformations=transformations,
        keypoint_csv_path=keypoint_csv_path,
        keypoint_output_dir=keypoint_output_dir,
    )

    # 顯示結果
    print("\n處理統計:")
    for name, stat in stats.items():
        print(f"{name}: 成功 {stat['success']}, 失敗 {stat['failed']}")
else:
    print(f"找不到輸入目錄: {input_dir}，請修改 input_dir 變數")


## Part 2

Below are additional transformation to bend skeletons by swirl effect.

Change root

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

def swirl_effect(
    img,
    angle=180,
    strength=1.0,
    center_x_ratio=0.4,
    center_y_ratio=0.5,
    keypoints=None,
):
    """
    旋轉扭曲效果：只在一定半徑內做 swirl，外圈保持原圖，避免邊界變成條紋。

    座標系: 原點在左上角, x 向右, y 向下 (和 OpenCV 影像座標一致)

    參數:
        img: 輸入圖片 (H, W, C, BGR格式)
        angle: 中心的最大旋轉角度（度數）
        strength: 扭曲強度，控制影響範圍 (0.0-2.0)
        center_x_ratio: 旋轉中心 X 位置比例 (0.0-1.0)
        center_y_ratio: 旋轉中心 Y 位置比例 (0.0-1.0)
        keypoints: (可選) numpy array, shape = (N, 2)，每列為 [x, y]，
                   表示「原圖座標」下的點。若提供，會一併輸出
                   swirl 之後對應到「輸出圖座標」下的新位置。

    回傳:
        若 keypoints is None:
            result_img
        否則:
            result_img, transformed_keypoints
    """
    height, width = img.shape[:2]

    # 旋轉中心（用我們約定的影像座標系）
    center_x = width * center_x_ratio
    center_y = height * center_y_ratio

    # 最大半徑（影響範圍）
    max_radius = min(width, height) * strength / 2.0
    if max_radius <= 0:
        if keypoints is None:
            return img.copy()
        else:
            # 沒有 swirl，圖 & 點都維持原樣
            return img.copy(), np.asarray(keypoints, dtype=np.float32).copy()

    # ------------------------------
    # 1) 生成圖像的 backward mapping (保持原本行為不變)
    # ------------------------------
    y, x = np.indices((height, width), dtype=np.float32)

    dx = x - center_x
    dy = y - center_y
    r = np.sqrt(dx**2 + dy**2)
    theta = np.arctan2(dy, dx)

    # 只在 r <= max_radius 的地方做 swirl
    mask = r <= max_radius

    # 正規化半徑 (0~1)
    r_normalized = np.zeros_like(r, dtype=np.float32)
    r_normalized[mask] = r[mask] / max_radius

    # 旋轉角度（距離越遠旋轉越小）: 這是「半徑函數」
    rotation_angle = np.zeros_like(r, dtype=np.float32)
    rotation_angle[mask] = angle * (1.0 - r_normalized[mask])

    rotation_rad = np.deg2rad(rotation_angle)
    new_theta = theta + rotation_rad  # 注意: 這是 output→input 的角度

    # 先預設為「不變形」
    x_new = x.copy()
    y_new = y.copy()

    # 只有在 mask 範圍內才改座標 (backward mapping: output -> input)
    x_new[mask] = center_x + r[mask] * np.cos(new_theta[mask])
    y_new[mask] = center_y + r[mask] * np.sin(new_theta[mask])

    # 避免極少數浮點誤差跑出邊界
    x_new = np.clip(x_new, 0, width - 1)
    y_new = np.clip(y_new, 0, height - 1)

    map_x = x_new.astype(np.float32)
    map_y = y_new.astype(np.float32)

    result = cv2.remap(
        img,
        map_x,
        map_y,
        cv2.INTER_LINEAR,
        borderMode=cv2.BORDER_REFLECT
    )

    # 如果沒有要處理 keypoints，就到此為止
    if keypoints is None:
        return result

    # ------------------------------
    # 2) 對 keypoints 做「正向」swirl 變換
    # ------------------------------
    kps = np.asarray(keypoints, dtype=np.float32).copy()
    kx = kps[:, 0]
    ky = kps[:, 1]

    # 相對中心的座標（同樣是左上原點，x 右 y 下）
    kdx = kx - center_x
    kdy = ky - center_y
    kr = np.sqrt(kdx**2 + kdy**2)
    ktheta = np.arctan2(kdy, kdx)

    transformed_kx = kx.copy()
    transformed_ky = ky.copy()

    # 只對在 swirl 半徑內的點做扭曲，外面保持不動
    k_mask = kr <= max_radius
    if np.any(k_mask):
        # 半徑正規化
        kr_norm = kr[k_mask] / max_radius

        # 注意：rotation_angle(r) 的定義要跟影像一致
        k_rotation_angle = angle * (1.0 - kr_norm)
        k_rotation_rad = np.deg2rad(k_rotation_angle)

        # 這邊是「正向 mapping」：原圖座標 (r, theta) -> output (r, theta_out)
        # 在你目前的 backward mapping 裡，output 的角度 + rotation → input 角度
        # => input_theta = output_theta + rotation
        # 反之，正向: output_theta = input_theta - rotation
        ktheta_out = ktheta[k_mask] - k_rotation_rad

        # 算回 Cartesian 座標 (x_out, y_out)
        transformed_kx[k_mask] = center_x + kr[k_mask] * np.cos(ktheta_out)
        transformed_ky[k_mask] = center_y + kr[k_mask] * np.sin(ktheta_out)

    # 邊界裁切，避免 round 後出界
    transformed_kx = np.clip(transformed_kx, 0, width - 1)
    transformed_ky = np.clip(transformed_ky, 0, height - 1)

    transformed_kps = np.stack([transformed_kx, transformed_ky], axis=1)

    return result, transformed_kps


In [ ]:
transformations = {
    "swirl_15": {
        "function": swirl_effect,
        "params": {
            "angle": 15,
            "strength": 1.0,
            "center_x_ratio": 0.4,
            "center_y_ratio": 0.5,
        },
    },
    "swirl_30": {
        "function": swirl_effect,
        "params": {
            "angle": 30,
            "strength": 1.0,
            "center_x_ratio": 0.4,
            "center_y_ratio": 0.5,
        },
    },
    "swirl_45": {
        "function": swirl_effect,
        "params": {
            "angle": 45,
            "strength": 1.0,
            "center_x_ratio": 0.4,
            "center_y_ratio": 0.5,
        },
    },
    "swirl_60": {
        "function": swirl_effect,
        "params": {
            "angle": 60,
            "strength": 1.0,
            "center_x_ratio": 0.4,
            "center_y_ratio": 0.5,
        },
    },
}

# ==== 路徑設定（請改成你實際的路徑）====
root = ""
# 骨架圖片所在資料夾
input_dir = root+"/sk/original"

# 變形後圖片的輸出根目錄（裡面會自動建立各個 transform 子資料夾）
output_dir = root+"/sk"

# 對應原始骨架圖片的 keypoints CSV
keypoint_csv_path = root+"/sk/original.csv"

# 變形後 keypoints CSV 的輸出資料夾
keypoint_output_dir = root+"/sk"

# 執行批次處理 (如果 input_dir 存在)
if os.path.exists(input_dir):
    print(f"開始處理目錄: {input_dir}")
    stats = batch_process_images(
        input_dir=input_dir,
        output_base_dir=output_dir,
        transformations=transformations,
        keypoint_csv_path=keypoint_csv_path,
        keypoint_output_dir=keypoint_output_dir,
    )

    # 顯示結果
    print("\n處理統計:")
    for name, stat in stats.items():
        print(f"{name}: 成功 {stat['success']}, 失敗 {stat['failed']}")
else:
    print(f"找不到輸入目錄: {input_dir}，請修改 input_dir 變數")

